In [1]:
import pickle
import collections

In [2]:
files = !ls ../data
files = [item for item in files if '.pickle' in item]
files

['CAO.pickle',
 'TAO.pickle',
 'data_D.pickle',
 'data_N.pickle',
 'data_zao.pickle',
 'sao_5district.pickle',
 'uao_9district.pickle']

In [3]:
hrefs_real = [ ]

for file in files:
    with open('../data/' + file, 'rb') as f:
        data_new = pickle.load(f)
    
    if str(type(data_new)) == "<class 'collections.defaultdict'>":
        hrefs = []
        for k,v in data_new.items():
            hrefs.extend(v) 
        hrefs_real += list(set(hrefs))
    else:
        hrefs_real += data_new # Файлы Димы
        
print(len(hrefs_real), len(set(hrefs_real)))

hrefs_real = list(set(hrefs_real))

30173 30037


In [4]:
import requests
from bs4 import BeautifulSoup
import re

import cian_info as cian

In [5]:
url = hrefs_real[0]
url

'https://www.cian.ru/sale/flat/224106732/'

Объединяем всё в одну функцию! 

In [6]:
def infa_download(soup):
    infa_dict = {}
    infa_dict.update(cian.get_general_information(soup))
    infa_dict.update(cian.get_about(soup))
    infa_dict.update(cian.get_price(soup))
    infa_dict.update(cian.get_description(soup))
    infa_dict.update(cian.to_info(soup))
    infa_dict.update(cian.get_address(soup))
    infa_dict.update(cian.get_metro(soup))
    infa_dict.update(cian.get_title(soup))
    infa_dict.update(cian.get_coord(soup))
    return infa_dict

In [10]:
import time
from requests.exceptions import ConnectionError

# all_infa = [ ]

for url in hrefs_real[15:]:
    try:
        time.sleep(1)
        soup = cian.get_soup(url)
    except ConnectionError:
        time.sleep(10)
        soup = cian.get_soup(url)
    
    if soup.find("div", {"data-name": "OfferUnpublished"}):
        print('объявление снято')
        continue
        
    cur_infa = infa_download(soup)
    all_infa.append(cur_infa)

KeyboardInterrupt: 

In [11]:
hrefs_real.index(url)

27

In [ ]:
len(all_infa)

In [12]:
import pandas as pd
df = pd.DataFrame(all_infa)
df.head()

,lat,long,Аварийность,Балкон/лоджия,Вид из окон,Высота потолков,Газоснабжение,Год постройки,Жилая,Кухня,...,Строительная серия,Тип дома,Тип жилья,Тип перекрытий,Этаж,адрес,метро,описание,тайтл,цена
0,55.774745,37.548609,Нет,1 балкон,NaN,3 м,NaN,2010,NaN,NaN,...,NaN,Монолитный,Вторичка,Железобетонные,13 из 37,"Москва, САО, р-н Хорошевский, Хорошевское ш., ...","[Беговая ⋅ <1 мин. на транспорте, Беговая ⋅ ...","Просторная, светлая, уютная 3-х комнатная квар...","3-комн. квартира, 87 м²",23500000
1,55.771342,37.61865,Нет,1 лоджия,NaN,NaN,NaN,2007,200 м²,20 м²,...,NaN,NaN,Вторичка,NaN,3 из 9,"Москва, ЦАО, р-н Тверской, Цветной бул., 15к2",[Цветной бульвар],Прекрасная квартира для большой семьи с хороши...,"5-комн. квартира, 249,1 м²",150000000
2,55.727724,37.572288,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Вторичка,NaN,5 из 13,"Москва, ЦАО, р-н Хамовники, ул. Усачева, 11Б","[Спортивная ⋅ 10 мин. пешком, Фрунзенская ⋅ ...","ЖК ""Садовые кварталы"". К продаже предлагается ...","3-комн. квартира, 149,5 м²",84000000
3,55.754618,37.524058,NaN,NaN,На улицу и двор,"3,1 м",NaN,NaN,"15,5 м²","12,6 м²",...,NaN,NaN,Вторичка,NaN,1 из 37,"Москва, ЦАО, р-н Пресненский, Шмитовский проез...","[Шелепиха ⋅ 5 мин. пешком, Деловой центр ⋅ 4...",Город в городе! Вторая очередь строительства. ...,"1-комн. квартира, 41,18 м²",11136240
4,55.715718,37.617905,Нет,NaN,На улицу и двор,NaN,Центральное,1929,"52,6 м²","6,8 м²",...,NaN,NaN,Вторичка,NaN,1 из 5,"Москва, ЮАО, р-н Даниловский, Городская ул., 5","[Шаболовская ⋅ 12 мин. пешком, Тульская ⋅ 13...","Продается теплая, уютная трехкомнатная квартир...","3-комн. квартира, 75 м²",13000000


In [13]:
df.shape

(27, 33)

In [14]:
df.isnull().sum()

lat                    0
long                   0
Аварийность            3
Балкон/лоджия          9
Вид из окон            7
Высота потолков       15
Газоснабжение         20
Год постройки          3
Жилая                  4
Кухня                  5
Лифты                  0
Мусоропровод          14
Общая                  0
Отделка               26
Отопление              5
Парковка              19
Планировка            16
Площадь комнат         9
Подъезды               4
Построен              13
Ремонт                 3
Санузел                1
Срок сдачи            25
Строительная серия    24
Тип дома               7
Тип жилья              0
Тип перекрытий         6
Этаж                   0
адрес                  0
метро                  0
описание               0
тайтл                  0
цена                   0
dtype: int64